<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/Lanngraph_corrective_rag_self_rag_and_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook we will try to create the production ready different RAG approaches that langgraph supports

### We will start with creating a corrective rag which will grade the document and generate the best answer

In [ ]:
pip install langchain langgraph langchain-core langchain-community langchain-groq langchain faiss-cpu langchain-huggingface

In [34]:
# let us understand how the document grader works first by creating the retriever using faiss

from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [35]:
loader=TextLoader("/content/nepal_insurance_details.txt")
docs=loader.load()

In [36]:
embeddings=HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

In [37]:
text_splitter=RecursiveCharacterTextSplitter(chunk_overlap=100, chunk_size=500)
chunk=text_splitter.split_documents(docs)
vector_stores=FAISS.from_documents(chunk, embeddings)

In [38]:
retriever=vector_stores.as_retriever(search_kwargs={"k":3})

In [39]:
result=retriever.invoke("What is the document about?")

In [40]:
print(result)

[Document(id='4a3c2a38-e711-457f-b2d7-9611e1776c3d', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Present Documents: Show your NHIP card (family or individual) and valid ID like citizenship certificate; register at the insurance desk if first-time.\nPay Co-Pay: Cover 20% of approved costs immediately (e.g., NPR 200 on a NPR 1,000 bill); no upfront payment for fully covered vulnerable groups like seniors over 70.\nGet Treatment: Receive services; hospital bills the HIB for the 80% reimbursement directly through IMIS no manual claim forms needed from you.'), Document(id='c198f58d-05bb-4475-9777-ec6a541d08cd', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Enrollment Procedure\nCitizens enroll via local Enrollment Assistants (volunteers) at ward offices, health posts, or empaneled facilities using the Insurance Management Information System (IMIS). Provide citizenship certificate, photos, and pay NPR 3,500 annually for a standard f

In [41]:
for doc in result:
  print(doc.id)
  print(doc.metadata)
  print(doc.page_content)
  print("\n\n-------")

4a3c2a38-e711-457f-b2d7-9611e1776c3d
{'source': '/content/nepal_insurance_details.txt'}
Present Documents: Show your NHIP card (family or individual) and valid ID like citizenship certificate; register at the insurance desk if first-time.
Pay Co-Pay: Cover 20% of approved costs immediately (e.g., NPR 200 on a NPR 1,000 bill); no upfront payment for fully covered vulnerable groups like seniors over 70.
Get Treatment: Receive services; hospital bills the HIB for the 80% reimbursement directly through IMIS no manual claim forms needed from you.


-------
c198f58d-05bb-4475-9777-ec6a541d08cd
{'source': '/content/nepal_insurance_details.txt'}
Enrollment Procedure
Citizens enroll via local Enrollment Assistants (volunteers) at ward offices, health posts, or empaneled facilities using the Insurance Management Information System (IMIS). Provide citizenship certificate, photos, and pay NPR 3,500 annually for a standard family (recent figure); receive a health insurance card instantly or shortly

## This is the data that we get from the retriever, now we will use this data to check if it is correct or not than only we will go with the generation ,hence this is the implementation of the corrective rag